In [1]:
import pandas as pd
import numpy as np

# Part I: Train classifiers

In [2]:
trainDf=pd.read_pickle('train.pkl')

In [4]:
trainDf.head()

,business_id,date,review_id,stars,text,user_id,active
467878,IZI9BdElfGB-WcHpd7Wg7Q,2014-02-20,CPj_ZWalJpnebGrFxV8ujg,5,Came here for a late lunch after being slightl...,PTcKYk_qU-WzbQjraE3TgQ,1
450196,Ja4stXdNYr39u5CZHMNtjw,2012-05-14,S1EKSngF9LVvFQ08Pye59Q,4,What a cute little place in Old Town! We went...,l5x4wzz9VOcQ22XMthU6uw,1
1295251,bjllusUsLq9zE4yZnLpOzg,2015-09-19,9ndOEDaRsYjETx0kl83h7w,3,Food was better than average. We got three dif...,76crp1vXF1A3LOoNP6zcUQ,1
379392,QdbwZGnBWdkdN2XAG7rwLA,2014-08-15,raKjAf09z4FtAqmqfb5lUw,5,Korean Style Fried Chicken has become the only...,4KbUaerUBoiUwZGMHKVFxQ,1
564809,WKOUTdVJS58E178JjhwidQ,2016-01-10,DQGL9V4vhF7wNBdVRcZ1VQ,4,"Cute little cafe, the crepe was so good. I tri...",XrYTMhY9YJvzX2pMepIz7A,1


In [6]:
X_train=trainDf['text'].values

In [10]:
Y_train=trainDf['active'].values

### Tokenize traning texts

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(339700, 180431)

In [43]:
type(X_train_counts)

scipy.sparse.csr.csr_matrix

### Count word frequency using tf-idf
*Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.
To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.
Another refinement on top of tf is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.
This downscaling is called tf–idf for “Term Frequency times Inverse Document Frequency”.*
————http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(339700, 180431)

In [59]:
X_train_tfidf

<339700x180431 sparse matrix of type '<class 'numpy.float64'>'
	with 26478750 stored elements in Compressed Sparse Row format>

In [63]:
count

{'norm': 'l2', 'smooth_idf': True, 'sublinear_tf': False, 'use_idf': True}

### Train a naive Bayes classifier

In [11]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, Y_train)

#### Evaluate the performance on the test set

In [13]:
testDf=pd.read_pickle('test.pkl')

In [14]:
testDf.head()

,business_id,date,review_id,stars,text,user_id,active
840771,LbyCngxEveE7G6w2mqeueg,2015-10-12,PhBoxX0S_htKMGweVmA7iA,4,"Delicious old school pizza, only deliver in a ...",UGlJYHK0I7TM6yLB3Rkp2w,1
524329,5N8R7ALESZ30EoAzVJtabw,2015-11-26,2ZLg2o_yh3pvXnQhdnBCFg,4,Probably like a lot of you that reviewed your ...,HgCv2lm-GuACRanD7bSqyA,1
1022586,hPO2di89hgfsD9inbKl7BQ,2014-12-03,z48GmttobtQ28hmKChpSRQ,4,This Ulta has greatly improved! The customer s...,HV5HZf7DOQHiwCtNdor2Zg,1
795885,5B-V6vr6TbtMjrRaBAtHqQ,2012-09-27,Y8jWjRX69baonQSXKLtPHg,2,"Not my favorite place to go, but I've only bee...",CPodThzsqATG_Bc8a4an_w,1
769454,u-SJ5QUwrNquL9VnXwl8cg,2013-08-08,UnZirwkho6bJkffEhSWlug,1,"I try to like you Postinos, I really do but yo...",k6vOe0_qglL2PZlBldoKeQ,1


In [16]:
X_test=testDf['text'].values

In [17]:
Y_test=testDf['active'].values

In [18]:
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [19]:
predicted = clf.predict(X_test_tfidf)

In [20]:
np.mean(predicted == Y_test)

0.77375597579068833

We achieved 77.4% accuracy.

### Train a linear support vector machine (SVM) model

In [24]:
from sklearn.linear_model import SGDClassifier
svm_clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None).fit(X_train_tfidf, Y_train)

In [49]:
svm_clf.coef_

array([[-0.10710299, -0.03974785, -0.00047589, ...,  0.        ,
        -0.00014326,  0.        ]])

#### Evaluate the performance on the test set

In [25]:
svm_predicted = svm_clf.predict(X_test_tfidf)

In [26]:
np.mean(svm_predicted == Y_test)

0.79949603183948381

In [31]:
from sklearn import metrics
print(metrics.classification_report(Y_test, svm_predicted,target_names=['inactive','active']))

             precision    recall  f1-score   support

   inactive       0.79      0.82      0.80     42811
     active       0.81      0.78      0.79     42115

avg / total       0.80      0.80      0.80     84926



### Parameter tuning using grid search

In [40]:
from sklearn.model_selection import GridSearchCV
parameters = {'alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(svm_clf, parameters)

In [44]:
gs_clf.fit(X_train_counts, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': (0.01, 0.001)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [45]:
gs_clf.best_params_

{'alpha': 0.01}

In [46]:
gs_clf.best_score_

0.826264350897851

In [50]:
svm_clf_tuned = SGDClassifier(loss='hinge', penalty='l2',alpha=0.01, random_state=42,max_iter=5, tol=None).fit(X_train_tfidf, Y_train)

In [53]:
svm_predicted_tuned = svm_clf_tuned.predict(X_test_tfidf)

In [54]:
np.mean(svm_predicted_tuned == Y_test)

0.73732425876645546

## Part II: Visualize & Analyze the result
Reference: https://buhrmann.github.io/tfidf-analysis.html

In [66]:
#holds a list of all the words in the tf-idf’s vocabulary
features=count_vect.get_feature_names()
print(features)

['00', '000', '0000', '00000', '0000000', '0000000000000001', '0000000003', '00001', '00007651', '0002', '0002136', '0003s', '0004318', '0004318zz', '000dollars', '000hr', '000iu', '000km', '000kms', '000lbs', '000mg', '000miles', '000s', '000sf', '000x', '001', '0015', '002', '003', '0030', '004', '005', '0051', '006', '0067', '007', '00719', '00789', '0089', '009', '00____', '00_____', '00______', '00a', '00am', '00dlls', '00dollars', '00each', '00eur', '00for', '00h', '00hh', '00hr', '00hrs', '00ish', '00lb', '00noon', '00off', '00or', '00p', '00pm', '00pmish', '00s', '00tip', '00uhr', '01', '010', '0105', '0109', '011856', '012', '0131', '0135', '013672', '014', '0143', '016', '017', '0176', '019', '01am', '01pm', '02', '0200', '0213', '0214', '0222000', '0233', '025', '0279', '0295', '02am', '02pm', '03', '030', '0300', '033', '0330', '0359', '03607', '0362', '0368', '0383', '03am', '03pm', '03rezepte', '04', '040', '0400', '040415', '041', '0430', '044', '04am', '04neue', '04pm',

Create a function that takes a single row of the tf-idf matrix (corresponding to a particular document), and return the n highest scoring words (or more generally tokens or features):

In [67]:
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

The result of a tf-idf, however, is typically a sparse matrix, which doesn’t support all the usual matrix or array operations. So in order to apply the above function to inspect a particular document, we convert a single row into dense format first:

In [69]:
def top_feats_in_doc(Xtr, features, row_id, top_n=25):
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features, top_n)

Using this to show the top 25 words used in the first review text, for example, which StumbleUpon has classified as ‘evergreen’, gives:

In [71]:
top_feats_in_doc(X_train_tfidf,features,0,10)

,feature,tfidf
0,yellow,0.242589
1,the,0.228426
2,was,0.225946
3,curry,0.200156
4,dahl,0.197804
5,whey,0.181877
6,it,0.164804
7,we,0.162261
8,hungover,0.160259
9,beef,0.157135


In [72]:
top_feats_in_doc(X_train_tfidf,features,1,10)

,feature,tfidf
0,football,0.257482
1,margaritas,0.247898
2,we,0.242036
3,were,0.241665
4,season,0.236595
5,scottsdale,0.210425
6,bunch,0.205950
7,specials,0.191353
8,sunday,0.186235
9,cute,0.186090


In [73]:
def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)

In [ ]:
top_mean_feats(X_train_tfidf,features)